In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm.autonotebook import tqdm
print(torch.__version__)
import pandas as pd
plt.ion()  

1.2.0


In [22]:
from sklearn.model_selection import train_test_split

In [19]:
def imread(path, fast_mode=False):
    img = cv2.imread(path)
    if not fast_mode and img is not None and len(img.shape) == 3:
        img = np.array(img[:, :, ::-1])
    return img

In [6]:
PATH = "./input/pku-autonomous-driving/"

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [20]:
train = pd.read_csv(PATH + "train.csv")

In [23]:
df_train, df_dev = train_test_split(train, test_size=0.01, random_state=42)

In [24]:
df_train

,ImageId,PredictionString
3046,ID_ffd457bf8,70 0.128341 -3.11313 -3.09698 -11.0544 3.7619 ...
505,ID_b839972b5,61 0.15791 0.149674 -3.12569 8.55243 5.09219 1...
1873,ID_012aef661,50 0.14442 -2.5374 -3.09288 -7.40432 10.9132 6...
817,ID_509457707,70 0.155235 -0.0271856 -3.13504 20.6082 8.4155...
166,ID_a9dd3799d,28 0.178162 -0.0101487 -3.09827 -3.37466 2.917...
3592,ID_e23272fdf,6 0.138208 0.016751 -3.13817 3.20057 2.78356 8...
584,ID_0aff7961d,31 0.142892 3.11488 -3.0951 -8.63689 4.0149 18...
4105,ID_d44bbe32c,40 0.15052 -0.00235959 -3.08098 4.33522 3.1731...
1192,ID_b11520d57,16 0.181339 -0.0192101 -3.11461 4.90094 3.8335...
1874,ID_126bd178a,61 0.188111 0.0133019 -3.08873 -3.31386 2.7680...
